<a href="https://colab.research.google.com/github/KacperGrodecki/image_proccessing/blob/main/kuba_img_text_reading_found_words_analyzing_enchant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install pyenchant

In [3]:
!apt install -qq enchant

enchant is already the newest version (1.6.0-11.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [4]:
!apt-get install myspell-pl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
myspell-pl is already the newest version (20170707-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [5]:
import pandas as pd
import cv2
import numpy as np
import re
import matplotlib.pyplot as plt      
import pickle
from os import listdir
from os.path import isfile, join
import gc
import sys
import pickle
import string
import enchant

In [6]:
d = enchant.Dict('PL')
d.check("guzik") # False



True

In [7]:
text=pd.read_csv('drive/MyDrive/kuba/odczyt_0_85.csv',index_col=0)

In [8]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [65]:
def word_correction_enchant(a):
    a=str(a)
    correct=''
    for word in a.split():
        if len(word)>1:
            if word.isnumeric() or len(word)<3:
              continue
            try:
              if int(d.check(word))==1:
                correct+=word.lower()+' '
            except:
              pass     
    return correct

In [66]:
correct=text.applymap(lambda x: word_correction_enchant(x))

In [83]:
correct.head(20)

,0
0,
1,własność archiwum tytuł teczki daty skrajne je...
2,ipn
3,karta informacyjna
4,instytut pamięci narodowej warszawie
5,
6,msw
7,tytuł teczki
8,pax- materiały dotyczące działalności zrzeszenia
9,daty skrajne


In [84]:
correct1 = correct[correct['0']!= '']

In [14]:
pip install spacy==2.3.2

In [15]:
!python3 -m spacy download pl_core_news_md
#https://github.com/explosion/spaCy/issues/5140

     |████████████████████████████████| 79.9 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('pl_core_news_md')


In [85]:
import spacy  
from spacy.lang.pl.examples import sentences 

In [86]:
nlp = spacy.load("pl_core_news_md")

In [87]:
df=correct1.copy()

In [88]:
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [89]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['0'])

In [90]:
from time import time

In [91]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.13 mins


In [92]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(4882, 1)

In [93]:
from gensim.models.phrases import Phrases, Phraser

In [94]:
sent = [row.split() for row in df_clean['clean']]

In [95]:
phrases = Phrases(sent, min_count=5, progress_per=100)

In [96]:
bigram = Phraser(phrases)

In [97]:
sentences = bigram[sent]

In [98]:
from collections import defaultdict  # For word frequency

In [99]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

5406

In [100]:
sorted(word_freq, key=word_freq.get, reverse=True)[:20]

['si',
 'pax',
 'y',
 'praca',
 'c',
 'dzia_alno',
 'sprawa',
 'e',
 'stowarzyszenie',
 'nale_y',
 'r',
 'ce',
 'swój',
 'dzień',
 'kt_re',
 'cz',
 'ca',
 'kraj',
 'd',
 'mo_e']

In [101]:
import multiprocessing

from gensim.models import Word2Vec

In [102]:
cores = multiprocessing.cpu_count()

In [103]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [104]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.0 mins


In [105]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.05 mins


In [106]:
w2v_model.init_sims(replace=True)

In [110]:
sorted(word_freq, key=word_freq.get, reverse=True)[:20]

['si',
 'pax',
 'y',
 'praca',
 'c',
 'dzia_alno',
 'sprawa',
 'e',
 'stowarzyszenie',
 'nale_y',
 'r',
 'ce',
 'swój',
 'dzień',
 'kt_re',
 'cz',
 'ca',
 'kraj',
 'd',
 'mo_e']

In [112]:
w2v_model.wv.most_similar(positive=["pax"])

[('kultura', 0.9899517893791199),
 ('si', 0.9895217418670654),
 ('wi', 0.9886922836303711),
 ('rozwój', 0.9870264530181885),
 ('y', 0.9869324564933777),
 ('g', 0.9866821765899658),
 ('polski', 0.9864038825035095),
 ('c', 0.9856300950050354),
 ('r_d', 0.9841951131820679),
 ('kt_re', 0.9836359024047852)]

In [113]:
w2v_model.wv.most_similar(negative=["pax"])

[('prowadzić', -0.4981674253940582),
 ('wa', -0.6150296926498413),
 ('wiedzieć', -0.7634900808334351),
 ('rozwi', -0.7882028222084045),
 ('prasa', -0.7916606068611145),
 ('chrze_cija', -0.8006922006607056),
 ('kontakt', -0.801023006439209),
 ('materia', -0.8052608966827393),
 ('socjalizm', -0.8238511085510254),
 ('wynikać', -0.8455989956855774)]

In [116]:
w2v_model.wv.doesnt_match(['pax', 'socjalizm', 'chrze_cija'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'chrze_cija'